`Weronika Sadzik`

`Python - zestaw 1`

`Analiza Numeryczna 2022/23`

In [1]:
import numpy as np
from decimal import *
from scipy.interpolate import interp1d
from typing import Union
import math

## Zadanie 1

Rozwiązać równania 

- $x^2 + 10^8x+1 \quad (1)$ 

- $x^2 - 10^8x+1 \quad (2)$,

używając szkolnych wzorów na rozwiązanie. Wypisać otrzymane $x_1, x_2$ i podstawić je z powrotem do równania. Jaki wynik otrzymaliśmy i dlaczego? Zaimplementować równoważny, numerycznie poprawny sposób na obliczenie pierwiastków równania kwadratowego na komputerze i porównać wyniki.

In [2]:
delta = 1e16 - 4

In [3]:
f1 = lambda x: x ** 2 + 1e8 * x + 1

x1 = (- 1e8 + delta ** .5) / 2
x2 = (- 1e8 - delta ** .5) / 2

print(x1, x2, '\n', sep='\n')
print(f1(x1), f1(x2), sep='\n')

-7.450580596923828e-09
-100000000.0


0.2549419403076172
1.0


In [4]:
f2 = lambda x: x ** 2 - 1e8 * x + 1

x1 = (1e8 + delta ** .5) / 2
x2 = (1e8 - delta ** .5) / 2

print(x1, x2, '\n', sep='\n')
print(f1(x1), f1(x2), sep='\n')

100000000.0
7.450580596923828e-09


2e+16
1.7450580596923828


Otrzymaliśmy wyniki różne od 0. Przyczyną tego faktu jest arytmetyka fl - konkretnie odejmowanie prawie równych liczb. W "szkolnym" wzorze na pierwiastki równania kwadratowego musimy odjąć $\sqrt{b^2-4ac}$ od $b$, które są prawie sobie równe - komputer nie jest w stanie przechować tak wielu miejsc po przecinku. 

Z drugiej strony, popatrzmy na same pierwiastki, które uzyskaliśmy - drugi pierwiastek jest bardzo bliski pierwiastkowi wyznaczonemu "na kartce", ale pierwszy z powodu reprezentacji liczb zmiennoprzecinkowych różni się znacząco od poprawnego rozwiązania.


Możemy skorzystać z innego algorytmu na pierwiastków równania kwadratowego, który jest stabilniejszy numerycznie:

$x_1 = \frac{-b-sgn(b)\sqrt{b^2-4ac}}{2a}$

$x_2 = \frac{2c}{-b-sgn(b)\sqrt{b^2-4ac}}$.

Istnieje również inny algorytm wyznaczania takich pierwiastków:

$x = \frac{2c}{-b\pm \sqrt{b^2-4ac}}$.

Warty zaznaczenia jest fakt, że nie istnieje jeden, uniwersalny algorytm - wszystko jest zależne od współczynników $a, b$ i $c$, ponieważ numerycznie możemy się w trakcie rozwiązywania takich równań na komputerze natchnąć na przeróżne problemy (innym przykładowym problemem jest bardzo mała wartość współczynnika $a$, gdyż zmuszamy wtedy komputer do dzielenia przez bardzo małą liczbę - ponownie wchodzi problem fl). 

In [5]:
a = 1
b = 1e8
#b = -1e8
c = 1

delta = b ** 2 - 4 * a * c 

In [6]:
u = -b - np.sign(b) * np.sqrt(delta)
x1 = u / (2 * a)
x2 = (2 * c) / u

print(x1)
print(x2)
print(f1(x1))
print(f1(x2))

-100000000.0
-1e-08
1.0
1.1102230246251565e-16


Wciąż nie jesteśmy w stanie otrzymać zera po podstawieniu uzyskanych pierwiastków do wzoru funkcji (ponownie kwestia arytmetyki fl), ale same pierwiastki, które otrzymaliśmy, są już bardzo bliskie pierwiatków wyznaczonych "na kartce".

Sprawdźmy jeszcze zachowanie drugiego algorytmu.

In [7]:
x1 = (2 * c) / (-b + math.sqrt(delta))
x2 = (2 * c) / (-b - math.sqrt(delta))
print(x1)
print(x2)
print(f1(x1))
print(f1(x2))

-134217728.0
-1e-08
4592625709481985.0
1.1102230246251565e-16


Również nie jesteśmy w stanie uzyskać idealnych wartości (i nigdy nie będziemy w stanie...), chociaż w tym przykładzie drugi pierwiastek po podstawieniu do wzoru funkcji bardzo mocno zbliżył się do 0, a pierwszy "eksplodował" oddalając się od rzeczywistego rozwiązania z wspomnianego wcześniej powodu odejmowania od siebie liczb, które są sobie prawie równe.

**Źródło algorytmu:** https://math.stackexchange.com/questions/4000135/quadratic-formula-fails-numerically-at-small-a-coefficients

Dodatkowo, chciałam sprawdzić, co się stanie, jeżeli skorzystamy z drugiego, stabilniejszego algorytmu, ale po użyciu dokładniejszej reprezentacji liczb zmiennoprzecinkowych na komputerze z użyciem biblioteki `decimal`. Ponieważ obliczenia nie są zbyt ciężkie obliczeniowo, możemy użyć bardzo wysokiej precyzji np. do 128 miejsc.

In [8]:
getcontext().prec = 128

In [9]:
a = 1
b = 1e8
#b = -1e8
c = 1

delta = Decimal(b ** 2 - 4 * a * c)

In [10]:
u = Decimal(-b) - Decimal(np.sign(b)) * Decimal(np.sqrt(delta))
x1 = u / Decimal(2 * a)
x2 = Decimal(2 * c) / u

print(x1)
print(x2)
print(f1(x1))
print(f1(x2))

-99999999.99999998999999999999999899999999999999979999999999999994999999999999998599999999999999579999999999999867999999999999957
-1.0000000000000001000000000000000200000000000000050000000000000014000000000000004200000000000001320000000000000429000000000000143E-8
-1E-112
0E-127


Zgodnie z przypuszczeniami, im większą precyzję ustawimy, tym wartość funkcji w wyznaczonym pierwiastku bardziej zbliża się do 0 i dla precyzji do 128 miejsc, po podstawieniu uzyskanych pierwiastków do wzoru funkcji, uzyskujemy liczby, które mają same zera do 110-125 miejsca po przecinku.

## Zadanie 8

IN: $f(x) = \cos x, x\in [0;10], x_1 = 1, x_2 = 2, \ldots, x_{10} = 10$.

OUT: Wykres wielomianu interpolacyjnego kawałkami liniowego, opartego na węzłach $x_1, \ldots, x_{10}$ (patrz poprzednie zadanie).

Proszę policzyć dla każdego z punktów z wektora `np.linspace(0, 10, 105)` błąd interpolacji. 

Wypisać minimum, maksimum i średnią z:

a) wektora różnic;

b) wektora błędów bezwzględnych dla wskazanych punktów.

Porównać swoje wyniki z błędami uzyskanymi poprzez przybliżenie funkcji $f$ przez obiekt `scipy.interpolate.interp1d`.

In [11]:
def spline_1d(x: Union[float, int], vals: list[tuple]) -> Union[float, int]:
    """
    Returns value at a given argument of linear interpolation function.
    If the argument is greater then last node (in sorted order) it returns value in the last node.
    If the argument is less then first node (in sorted order) it returns value in the first node.
    Args:
        x (Union[float, int]): argument in which we want to obtain interpolation function value
        vals_vect (list[tuple]): list of tuples, where first values are nodes 
                                 and second values are values of function in nodes
    """
    vals = sorted(vals, key=lambda tup: tup[0])
    n = len(vals)
    if x < vals[0][0]:
        return vals[0][1]
    
    if x > vals[n - 1][0]:
        return vals[n - 1][1]
    
    val1 = next((arg for arg in vals if arg[0] >= x), None)
    val0 = vals[vals.index(val1) - 1]
    y1, x1 = val1[1], val1[0]
    y0, x0 = val0[1], val0[0]
    yp = y0 + ((y1 - y0) / (x1 - x0)) * (x - x0)
    
    return yp

In [12]:
node_vals = [(i, np.math.cos(i)) for i in range(1, 11)]

In [13]:
interp_errors = np.array(list(map(lambda x: spline_1d(x, node_vals), np.linspace(0, 10, 105)))) - np.array(list(map(np.math.cos, 
                                                                                                    np.linspace(0, 10, 105))))

print(f'Minimum z wektora różnic: {np.min(interp_errors)}', f'Maksimum z wektora różnic: {np.max(interp_errors)}', 
      f'Średnia z wektora różnic: {np.mean(interp_errors)}', '\n', sep='\n')

interp_abs_errors = np.abs(interp_errors)
print(f'Minimum z wektora błędów bezwzględnych: {np.min(interp_abs_errors)}', f'Maksimum z błędów bezwzględnych: {np.max(interp_abs_errors)}', 
      f'Średnia z błędów bezwzględnych: {np.mean(interp_abs_errors)}', sep='\n')

Minimum z wektora różnic: -0.45969769413186023
Maksimum z wektora różnic: 0.12182749813960936
Średnia z wektora różnic: -0.020419828442142362


Minimum z wektora błędów bezwzględnych: 0.0
Maksimum z błędów bezwzględnych: 0.45969769413186023
Średnia z błędów bezwzględnych: 0.07765593848212136


In [14]:
x = [tup[0] for tup in node_vals]
y = [tup[1] for tup in node_vals]

In [15]:
interp_errors = np.array(list(map(interp1d(x, y, fill_value=(y[0], y[-1]), bounds_error=False), np.linspace(0, 10, 105)))) - np.array(list(map(np.math.cos, 
                                                                                                                 np.linspace(0, 10, 105))))
    
print(f'Minimum z wektora różnic: {np.min(interp_errors)}', f'Maksimum z wektora różnic: {np.max(interp_errors)}', 
      f'Średnia z wektora różnic: {np.mean(interp_errors)}', '\n', sep='\n')

interp_abs_errors = np.abs(interp_errors)
print(f'Minimum z wektora błędów bezwzględnych: {np.min(interp_abs_errors)}', f'Maksimum z błędów bezwzględnych: {np.max(interp_abs_errors)}', 
      f'Średnia z błędów bezwzględnych: {np.mean(interp_abs_errors)}', sep='\n')

Minimum z wektora różnic: -0.45969769413186023
Maksimum z wektora różnic: 0.12182749813960936
Średnia z wektora różnic: -0.020419828442142362


Minimum z wektora błędów bezwzględnych: 0.0
Maksimum z błędów bezwzględnych: 0.45969769413186023
Średnia z błędów bezwzględnych: 0.07765593848212136


**Wniosek:** wyniki pochodzące z obiektu `scipy.interpolate.interp1d` są identyczne, jak wyniki pochodzące z funkcji `spline_1d`.